In [26]:
from sympy import *
import numpy as np

### Anisotropic Drucker-Prager incompressible with nice simplification + out of plane!

In [27]:
# 1) Define plasticity model
a_p, a_ve = symbols('a_p, a_ve')

Diso = Matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, a_ve**-2, 0], [0, 0, 0, 0, a_ve**-2]])
iDpl = Matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, a_p**2,   0], [0, 0, 0, 0, a_p**2  ]])  # error 1
eta_ve, gdot, P, phi, Coh, half = symbols('eta_ve, gdot, P, fric, Coh, half')
a1, a2, a3 = symbols('a1, a2, a3')
Exx, Eyy, Ezz, Exy, Eyx = symbols('Exx, Eyy, Ezz, Exy, Eyx')
Eps = Matrix([[Exx], [Eyy], [Ezz], [Exy], [Eyx]])
Tau = eta_ve*Diso*Eps
Txx, Tyy, Tzz, Txy, Tyx = symbols('Txx, Tyy, Tzz, Txy, Tyx')
Tau = Matrix([[Txx], [Tyy], [Tzz], [Txy], [Tyx]])
J2  =  Rational(1,2)*(Tau.transpose() * iDpl * Tau)[0]
Ty  = Coh + P*sin(phi)/3*(a1+a2+a3) -  sin(phi)/3*( a1*Txx + a2*Tyy + a3*Tzz)
F   = sqrt( J2 ) - Ty 
display(F.subs(J2, 'J2'))
display(J2)

-Coh + sqrt(J2) - P*(a1 + a2 + a3)*sin(fric)/3 + (Txx*a1 + Tyy*a2 + Tzz*a3)*sin(fric)/3

Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2

In [28]:
F.diff(Txx)

Txx/(2*sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2)) + a1*sin(fric)/3

In [37]:
# 2) express corrected stress invariant
Eps_p =  gdot * Matrix([[F.diff(Txx)], [F.diff(Tyy)], [F.diff(Tzz)], [F.diff(Txy)], [F.diff(Tyx)] ])
# dFdTxx, dFdTyy, dFdTzz, dFdTxy, dFdTyx = symbols('dFdTxx, dFdTyy, dFdTzz, dFdTxy, dFdTyx')
# Eps_p =  gdot * Matrix([dFdTxx, dFdTyy, dFdTzz, dFdTxy, dFdTyx ])
display(Eps_p)
# Corrected stress
Tau_corr = Tau - 2*eta_ve*Diso*(Eps_p) # Tau      = eta_ve*Diso*(Eps - Eps_p)
# Simplify: factor out dev. stress components 
for i in range(5):
    Tau_corr[i] = collect(Tau_corr[i], Tau[i])
# Invariant
J2_corr  = Rational(1,2)*(Tau_corr.transpose() * iDpl * Tau_corr)[0]
# Simplify: collect out common factor
a = ( (1-(2*eta_ve*gdot)/sqrt(J2))**2 )
J2_corr1 = collect(J2_corr, a)
display(J2_corr1)
julia_code(J2_corr1)


Matrix([
[gdot*(Txx/(2*sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2)) + a1*sin(fric)/3)],
[gdot*(Tyy/(2*sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2)) + a2*sin(fric)/3)],
[gdot*(Tzz/(2*sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2)) + a3*sin(fric)/3)],
[            Txy*a_p**2*gdot/(2*sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2))],
[            Tyx*a_p**2*gdot/(2*sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2))]])

Txy**2*a_p**2*(-a_p**2*eta_ve*gdot/(a_ve**2*sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2)) + 1)**2/2 + Tyx**2*a_p**2*(-a_p**2*eta_ve*gdot/(a_ve**2*sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2)) + 1)**2/2 + (Txx - 2*eta_ve*gdot*(Txx/(2*sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2)) + a1*sin(fric)/3))**2/2 + (Tyy - 2*eta_ve*gdot*(Tyy/(2*sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2)) + a2*sin(fric)/3))**2/2 + (Tzz - 2*eta_ve*gdot*(Tzz/(2*sqrt(Txx**2/2 + Txy**2*a_p**2/2 + Tyx**2*a_p**2/2 + Tyy**2/2 + Tzz**2/2)) + a3*sin(fric)/3))**2/2

'Txy .^ 2 .* a_p .^ 2 .* (-a_p .^ 2 .* eta_ve .* gdot ./ (a_ve .^ 2 .* sqrt(Txx .^ 2 / 2 + Txy .^ 2 .* a_p .^ 2 / 2 + Tyx .^ 2 .* a_p .^ 2 / 2 + Tyy .^ 2 / 2 + Tzz .^ 2 / 2)) + 1) .^ 2 / 2 + Tyx .^ 2 .* a_p .^ 2 .* (-a_p .^ 2 .* eta_ve .* gdot ./ (a_ve .^ 2 .* sqrt(Txx .^ 2 / 2 + Txy .^ 2 .* a_p .^ 2 / 2 + Tyx .^ 2 .* a_p .^ 2 / 2 + Tyy .^ 2 / 2 + Tzz .^ 2 / 2)) + 1) .^ 2 / 2 + (Txx - 2 * eta_ve .* gdot .* (Txx ./ (2 * sqrt(Txx .^ 2 / 2 + Txy .^ 2 .* a_p .^ 2 / 2 + Tyx .^ 2 .* a_p .^ 2 / 2 + Tyy .^ 2 / 2 + Tzz .^ 2 / 2)) + a1 .* sin(fric) / 3)) .^ 2 / 2 + (Tyy - 2 * eta_ve .* gdot .* (Tyy ./ (2 * sqrt(Txx .^ 2 / 2 + Txy .^ 2 .* a_p .^ 2 / 2 + Tyx .^ 2 .* a_p .^ 2 / 2 + Tyy .^ 2 / 2 + Tzz .^ 2 / 2)) + a2 .* sin(fric) / 3)) .^ 2 / 2 + (Tzz - 2 * eta_ve .* gdot .* (Tzz ./ (2 * sqrt(Txx .^ 2 / 2 + Txy .^ 2 .* a_p .^ 2 / 2 + Tyx .^ 2 .* a_p .^ 2 / 2 + Tyy .^ 2 / 2 + Tzz .^ 2 / 2)) + a3 .* sin(fric) / 3)) .^ 2 / 2'

In [30]:
Jii, Ji  = symbols('Jii, Ji')
g    = J2_corr1.subs(J2, Jii)
ap_n = (1 - eta_ve*gdot/sqrt(Jii))
ap_s = (1 - a_p**2*eta_ve*gdot/(sqrt(Jii)*a_ve**2))
print(g.subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' ))

Txy**2*a_p**2*ap_s**2/2 + Tyx**2*a_p**2*ap_s**2/2 + (Txx - 2*eta_ve*gdot*(a1*sin(fric)/3 + Txx/(2*sqrt(Jii))))**2/2 + (Tyy - 2*eta_ve*gdot*(a2*sin(fric)/3 + Tyy/(2*sqrt(Jii))))**2/2 + (Tzz - 2*eta_ve*gdot*(a3*sin(fric)/3 + Tzz/(2*sqrt(Jii))))**2/2


In [31]:
J1      = a1*Txx + a2*Tyy + a3*Tzz
Txxc    = Tau_corr[0]
Tyyc    = Tau_corr[1]
Tzzc    = -Txxc-Tyyc
J1_corr = a1*Txxc + a2*Tyyc + a3*Tzzc


In [32]:
K, dt, psi, eta_vp = symbols('K, dt, dil, eta_vp')
dQdP    = -sin(psi)*Rational(1,3)*(a1+a2+a3)
Pc      = P - K*dt*gdot*dQdP
Ty      = Coh + Pc*sin(phi)*Rational(1,3)*(a1+a2+a3)  -  sin(phi)*Rational(1,3)*J1_corr + eta_vp*gdot
F_corr  = sqrt(J2_corr1) - Ty
dFdgdot = F_corr.diff(gdot)
print('#----------- JULIA VERSIONS -----------#')
print('ap_n = ' + julia_code(ap_n) + ';')
print()
print('ap_s = ' + julia_code(ap_s) + ';')
print()
print('J1_corr = ' + julia_code(J1_corr.subs(J2, Jii).subs(J1, Ji).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' )) + ';')
print()
print('J2_corr = ' + julia_code(J2_corr1.subs(J2, Jii).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' )) + ';')
print()
print('Fc = ' + julia_code(F_corr.subs(J2_corr, 'J2_corr').subs(J1_corr, 'J1_corr').subs(J1, 'J1').subs(Pc,'Pc')) + ';')
print()
J2_corr_subs = J2_corr1.subs(J2, Jii).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' )
print('dFdgdot = ' + julia_code(dFdgdot.subs(J2, Jii).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' ).subs(J2_corr_subs,'J2_corr').subs(J1_corr, 'J1_corr').subs(J1, 'J1')) + ';')





#----------- JULIA VERSIONS -----------#
ap_n = 1 - eta_ve .* gdot ./ sqrt(Jii);

ap_s = 1 - a_p .^ 2 .* eta_ve .* gdot ./ (sqrt(Jii) .* a_ve .^ 2);

J1_corr = a1 .* (Txx - 2 * eta_ve .* gdot .* (a1 .* sin(fric) / 3 + Txx ./ (2 * sqrt(Jii)))) + a2 .* (Tyy - 2 * eta_ve .* gdot .* (a2 .* sin(fric) / 3 + Tyy ./ (2 * sqrt(Jii)))) + a3 .* (-Txx - Tyy + 2 * eta_ve .* gdot .* (a1 .* sin(fric) / 3 + Txx ./ (2 * sqrt(Jii))) + 2 * eta_ve .* gdot .* (a2 .* sin(fric) / 3 + Tyy ./ (2 * sqrt(Jii))));

J2_corr = Txy .^ 2 .* a_p .^ 2 .* ap_s .^ 2 / 2 + Tyx .^ 2 .* a_p .^ 2 .* ap_s .^ 2 / 2 + (Txx - 2 * eta_ve .* gdot .* (a1 .* sin(fric) / 3 + Txx ./ (2 * sqrt(Jii)))) .^ 2 / 2 + (Tyy - 2 * eta_ve .* gdot .* (a2 .* sin(fric) / 3 + Tyy ./ (2 * sqrt(Jii)))) .^ 2 / 2 + (Tzz - 2 * eta_ve .* gdot .* (a3 .* sin(fric) / 3 + Tzz ./ (2 * sqrt(Jii)))) .^ 2 / 2;

Fc = -Coh + J1_corr .* sin(fric) / 3 + sqrt(J2_corr) - Pc .* (a1 + a2 + a3) .* sin(fric) / 3 - eta_vp .* gdot;

dFdgdot = -K .* dt .* (a1 + a2 + a3) .^ 2

In [33]:
display(dFdgdot.subs(J2, Jii).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' ).subs(J2_corr_subs,'J2_corr').subs(J1_corr, 'J1_corr').subs(J1, 'J1'))

-K*dt*(a1 + a2 + a3)**2*sin(dil)*sin(fric)/9 - eta_vp + (-2*a1*eta_ve*(a1*sin(fric)/3 + Txx/(2*sqrt(Jii))) - 2*a2*eta_ve*(a2*sin(fric)/3 + Tyy/(2*sqrt(Jii))) + a3*(2*eta_ve*(a1*sin(fric)/3 + Txx/(2*sqrt(Jii))) + 2*eta_ve*(a2*sin(fric)/3 + Tyy/(2*sqrt(Jii)))))*sin(fric)/3 + (-eta_ve*(Txx - 2*eta_ve*gdot*(a1*sin(fric)/3 + Txx/(2*sqrt(Jii))))*(a1*sin(fric)/3 + Txx/(2*sqrt(Jii))) - eta_ve*(Tyy - 2*eta_ve*gdot*(a2*sin(fric)/3 + Tyy/(2*sqrt(Jii))))*(a2*sin(fric)/3 + Tyy/(2*sqrt(Jii))) - eta_ve*(Tzz - 2*eta_ve*gdot*(a3*sin(fric)/3 + Tzz/(2*sqrt(Jii))))*(a3*sin(fric)/3 + Tzz/(2*sqrt(Jii))) - Txy**2*a_p**4*ap_s*eta_ve/(2*sqrt(Jii)*a_ve**2) - Tyx**2*a_p**4*ap_s*eta_ve/(2*sqrt(Jii)*a_ve**2))/sqrt(J2_corr)

In [34]:
print('//----------- C VERSIONS -----------')
Y1_p, Y2_p = symbols('Y1_p,Y2_p')
Y1_p_c = symbols('Y1_p_c')
print('ap_n = ' + ccode(ap_n.subs(Jii,Y2_p)) + ';')
print()
print('ap_s = ' + ccode(ap_s.subs(Jii,Y2_p)) + ';')
print()
print('Y1_p_c = ' + ccode(J1_corr.subs(J2, Jii).subs(J1, Ji).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' ).subs(Txx,'txx').subs(Tyy,'tzz').subs(Tzz,'tyy').subs(Txy,'txz').subs(Tyx,'txz')) + ';')
print()
print('Y2_p_c = ' + ccode(J2_corr1.subs(J2, Jii).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' ).subs(Txx,'txx').subs(Tyy,'tzz').subs(Tzz,'tyy').subs(Txy,'txz').subs(Tyx,'txz')) + ';')
print()
print('Fc = ' + ccode(F_corr.subs(J2_corr, 'Y2_p_c').subs(J1_corr, 'Y1_p_c').subs(J1, 'J1').subs(Pc,'Pc')) + ';')
print()
J2_corr_subs = J2_corr1.subs(J2, Jii).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' )
print('dFdgdot = ' + ccode(dFdgdot.subs(J2, Jii).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' ).subs(J2_corr_subs,'Y2_p_c').subs(J1_corr, 'J1_corr').subs(J1, 'J1').subs(J2_corr,'Y2_p_c').subs(Jii, 'Y2_p').subs(Txx,'txx').subs(Tyy,'tzz').subs(Tzz,'tyy').subs(Txy,'txz').subs(Tyx,'txz')) + ';')

//----------- C VERSIONS -----------
ap_n = 1 - eta_ve*gdot/sqrt(Y2_p);

ap_s = 1 - pow(a_p, 2)*eta_ve*gdot/(sqrt(Y2_p)*pow(a_ve, 2));

Y1_p_c = a1*(-2*eta_ve*gdot*((1.0/3.0)*a1*sin(fric) + (1.0/2.0)*txx/sqrt(Jii)) + txx) + a2*(-2*eta_ve*gdot*((1.0/3.0)*a2*sin(fric) + (1.0/2.0)*tzz/sqrt(Jii)) + tzz) + a3*(2*eta_ve*gdot*((1.0/3.0)*a1*sin(fric) + (1.0/2.0)*txx/sqrt(Jii)) + 2*eta_ve*gdot*((1.0/3.0)*a2*sin(fric) + (1.0/2.0)*tzz/sqrt(Jii)) - txx - tzz);

Y2_p_c = pow(a_p, 2)*pow(ap_s, 2)*pow(txz, 2) + (1.0/2.0)*pow(-2*eta_ve*gdot*((1.0/3.0)*a1*sin(fric) + (1.0/2.0)*txx/sqrt(Jii)) + txx, 2) + (1.0/2.0)*pow(-2*eta_ve*gdot*((1.0/3.0)*a2*sin(fric) + (1.0/2.0)*tzz/sqrt(Jii)) + tzz, 2) + (1.0/2.0)*pow(-2*eta_ve*gdot*((1.0/3.0)*a3*sin(fric) + (1.0/2.0)*tyy/sqrt(Jii)) + tyy, 2);

Fc = -Coh - 1.0/3.0*Pc*(a1 + a2 + a3)*sin(fric) + (1.0/3.0)*Y1_p_c*sin(fric) + sqrt(Y2_p_c) - eta_vp*gdot;

dFdgdot = -1.0/9.0*K*dt*pow(a1 + a2 + a3, 2)*sin(dil)*sin(fric) - eta_vp + (1.0/3.0)*(-2*a1*eta_ve*((1.0/3.0)*a1*s

In [35]:
print('//----------- J VERSIONS -----------')
Y1_p, Y2_p = symbols('Y1_p,Y2_p')
Y1_p_c = symbols('Y1_p_c')
print('ap_n = ' + julia_code(ap_n.subs(Jii,Y2_p)) + ';')
print()
print('ap_s = ' + julia_code(ap_s.subs(Jii,Y2_p)) + ';')
print()
print('Y1_p_c = ' + julia_code(J1_corr.subs(J2, Jii).subs(J1, Ji).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' ).subs(Txx,'Txx').subs(Tyy,'Tyy').subs(Tzz,'Tzz').subs(Txy,'Txy').subs(Tyx,'Txy')) + ';')
print()
print('Y2_p_c = ' + julia_code(J2_corr1.subs(J2, Jii).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' ).subs(Txx,'Txx').subs(Tyy,'Tyy').subs(Tzz,'Tzz').subs(Txy,'Txy').subs(Tyx,'Txy')) + ';')
print()
print('Fc = ' + julia_code(F_corr.subs(J2_corr, 'Y2_p_c').subs(J1_corr, 'Y1_p_c').subs(J1, 'J1').subs(Pc,'Pc')) + ';')
print()
J2_corr_subs = J2_corr1.subs(J2, Jii).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' )
print('dFdgdot = ' + julia_code(dFdgdot.subs(J2, Jii).subs(ap_n, 'ap_n' ).subs(ap_s, 'ap_s' ).subs(J2_corr_subs,'Y2_p_c').subs(J1_corr, 'J1_corr').subs(J1, 'J1').subs(J2_corr,'Y2_p_c').subs(Jii, 'Y2_p').subs(Txx,'txx').subs(Tyy,'tzz').subs(Tzz,'tyy').subs(Txy,'txz').subs(Tyx,'txz')) + ';')

//----------- J VERSIONS -----------
ap_n = 1 - eta_ve .* gdot ./ sqrt(Y2_p);

ap_s = 1 - a_p .^ 2 .* eta_ve .* gdot ./ (sqrt(Y2_p) .* a_ve .^ 2);

Y1_p_c = a1 .* (Txx - 2 * eta_ve .* gdot .* (a1 .* sin(fric) / 3 + Txx ./ (2 * sqrt(Jii)))) + a2 .* (Tyy - 2 * eta_ve .* gdot .* (a2 .* sin(fric) / 3 + Tyy ./ (2 * sqrt(Jii)))) + a3 .* (-Txx - Tyy + 2 * eta_ve .* gdot .* (a1 .* sin(fric) / 3 + Txx ./ (2 * sqrt(Jii))) + 2 * eta_ve .* gdot .* (a2 .* sin(fric) / 3 + Tyy ./ (2 * sqrt(Jii))));

Y2_p_c = Txy .^ 2 .* a_p .^ 2 .* ap_s .^ 2 + (Txx - 2 * eta_ve .* gdot .* (a1 .* sin(fric) / 3 + Txx ./ (2 * sqrt(Jii)))) .^ 2 / 2 + (Tyy - 2 * eta_ve .* gdot .* (a2 .* sin(fric) / 3 + Tyy ./ (2 * sqrt(Jii)))) .^ 2 / 2 + (Tzz - 2 * eta_ve .* gdot .* (a3 .* sin(fric) / 3 + Tzz ./ (2 * sqrt(Jii)))) .^ 2 / 2;

Fc = -Coh - Pc .* (a1 + a2 + a3) .* sin(fric) / 3 + Y1_p_c .* sin(fric) / 3 + sqrt(Y2_p_c) - eta_vp .* gdot;

dFdgdot = -K .* dt .* (a1 + a2 + a3) .^ 2 .* sin(dil) .* sin(fric) / 9 - eta_vp + (-2 * a1 

In [36]:
# try Picard